In [20]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

precinct_df = pd.read_csv('../data/mi_matched_precinct_classifier.csv')
precinct_df.columns

Index(['total_pop', 'pop_density', 'prop_white', 'prop_Black', 'prop_Hispanic',
       'prop_AIA', 'prop_Asian', 'prop_married', 'prop_nonfamily',
       'prop_family_nonmarried', 'prop_hs_diploma', 'prop_some_college',
       'prop_bachelors', 'prop_higher', 'prop_under_hs', 'prop_under_30k',
       'prop_30_to_50k', 'prop_50_to_100k', 'prop_more_than_100k',
       'D_pct_vote', 'Weighted_ideology', 'Winner', 'Partisanship',
       'Aggregate'],
      dtype='object')

In [21]:
precinct_df['Winner'] = precinct_df['Winner'].astype('string')
precinct_df.head()

,total_pop,pop_density,prop_white,prop_Black,prop_Hispanic,prop_AIA,prop_Asian,prop_married,prop_nonfamily,prop_family_nonmarried,...,prop_under_hs,prop_under_30k,prop_30_to_50k,prop_50_to_100k,prop_more_than_100k,D_pct_vote,Weighted_ideology,Winner,Partisanship,Aggregate
0,129.536026,0.000005,0.970255,0.000000,0.012748,0.016997,0.000000,0.588889,0.369444,0.041667,...,0.666116,0.344444,0.247222,0.266667,0.141667,0.228571,-0.078343,Republican,Split,"Republican,Split"
1,460.331798,0.000016,0.970038,0.000000,0.013012,0.016950,0.000000,0.587382,0.370388,0.042230,...,0.666143,0.344266,0.246589,0.267258,0.141887,0.215116,0.157657,Republican,Split,"Republican,Split"
2,656.081894,0.000013,0.987864,0.000000,0.007282,0.004854,0.000000,0.696721,0.237705,0.065574,...,0.731544,0.209016,0.178279,0.461066,0.151639,0.183246,-0.031272,Republican,Split,"Republican,Split"
3,634.000000,0.000007,0.973186,0.000000,0.017350,0.009464,0.000000,0.553719,0.309917,0.136364,...,0.801418,0.227273,0.285124,0.347107,0.140496,0.224409,-0.142366,Republican,Split,"Republican,Split"
4,1067.006633,0.001649,0.929709,0.003749,0.001875,0.000000,0.031865,0.710213,0.247031,0.042756,...,0.411376,0.168646,0.114014,0.315915,0.401425,0.344262,-0.132125,Republican,Close,"Republican,Close"


In [22]:
precinct_df['winner_category'] = 2
precinct_df.loc[(precinct_df['Winner'] == 'Republican'), 'winner_category'] = 0
precinct_df.loc[(precinct_df['Winner'] == 'Democrat'), 'winner_category'] = 1

precinct_df['wtd_ideology_cat'] = 0
precinct_df.loc[(precinct_df['Weighted_ideology'] < -1), 'wtd_ideology_cat'] = 1
precinct_df.loc[(precinct_df['Weighted_ideology'] >= -1) & (precinct_df['Weighted_ideology'] <= -0.5), 'wtd_ideology_cat'] = 2
precinct_df.loc[(precinct_df['Weighted_ideology'] >= -0.5) & (precinct_df['Weighted_ideology'] < 0), 'wtd_ideology_cat'] = 3
precinct_df.loc[(precinct_df['Weighted_ideology'] >= 0) & (precinct_df['Weighted_ideology'] < 0.5), 'wtd_ideology_cat'] = 4
precinct_df.loc[(precinct_df['Weighted_ideology'] >= 0.5) & (precinct_df['Weighted_ideology'] < 1), 'wtd_ideology_cat'] = 5
precinct_df.loc[(precinct_df['Weighted_ideology'] >= 1), 'wtd_ideology_cat'] = 6

precinct_df['partisanship_category'] = 2
precinct_df.loc[(precinct_df['Partisanship'] == 'Close'), 'partisanship_category'] = 1
precinct_df.loc[(precinct_df['Winner'] == 'Split'), 'partisanship_category'] = 0

In [23]:
precinct_df = precinct_df.drop(['D_pct_vote', 'Weighted_ideology', 'Winner', 'Partisanship', 'Aggregate'], axis = 'columns')
precinct_df = precinct_df.drop(['prop_white', "prop_nonfamily", "prop_higher", "prop_more_than_100k"], axis = 1)
precinct_df.head()

,total_pop,pop_density,prop_Black,prop_Hispanic,prop_AIA,prop_Asian,prop_married,prop_family_nonmarried,prop_hs_diploma,prop_some_college,prop_bachelors,prop_under_hs,prop_under_30k,prop_30_to_50k,prop_50_to_100k,winner_category,wtd_ideology_cat,partisanship_category
0,129.536026,0.000005,0.000000,0.012748,0.016997,0.000000,0.588889,0.041667,0.0,0.158678,0.090909,0.666116,0.344444,0.247222,0.266667,0,3,2
1,460.331798,0.000016,0.000000,0.013012,0.016950,0.000000,0.587382,0.042230,0.0,0.158594,0.091005,0.666143,0.344266,0.246589,0.267258,0,4,2
2,656.081894,0.000013,0.000000,0.007282,0.004854,0.000000,0.696721,0.065574,0.0,0.137584,0.097315,0.731544,0.209016,0.178279,0.461066,0,3,2
3,634.000000,0.000007,0.000000,0.017350,0.009464,0.000000,0.553719,0.136364,0.0,0.099291,0.078014,0.801418,0.227273,0.285124,0.347107,0,3,2
4,1067.006633,0.001649,0.003749,0.001875,0.000000,0.031865,0.710213,0.042756,0.0,0.152117,0.250000,0.411376,0.168646,0.114014,0.315915,0,3,1


In [24]:
precinct_df['wtd_ideology_cat'] = precinct_df['wtd_ideology_cat'].astype(int)
precinct_df['winner_category'].value_counts(dropna = False)

1    142
0    108
Name: winner_category, dtype: int64

In [25]:
precinct_arr = precinct_df.to_numpy()

In [26]:
training, testing = train_test_split(precinct_arr)
training_x, training_y = training[:,:-3], training[:,-3:].astype('int')
testing_x, testing_y = testing[:,:-3], testing[:,-3:]


logreg = MultiOutputClassifier(LogisticRegression(multi_class='multinomial', solver='lbfgs'))
logreg.fit(training_x, training_y)

MultiOutputClassifier(estimator=LogisticRegression(multi_class='multinomial'))

## Mean accuracy on the training and testing data and labels. 
 Checks for complete accuracy between classification vectors

In [28]:
logreg.score(training_x, training_y)

0.5828877005347594

In [29]:
logreg.score(testing_x, testing_y)

0.47619047619047616

## Value-wise accuracy
Checks for accuracy per value in classification vector and returns the percentage correct. 

Ex: If overlap is 2/3 in one records true vector of values and predicted, returns percentage of 66%

In [31]:
train_acc = np.count_nonzero(logreg.predict(training_x) == training_y) / (3*len(training_y))
print(train_acc)

test_y_hat = logreg.predict(testing_x)
test_acc =  np.count_nonzero(logreg.predict(testing_x) == testing_y) / (3*len(testing_y))
print(test_acc)

0.8502673796791443
0.8148148148148148
